# Argus

Scope:
* Sweden:
* Scandinavia:
* Europe:
* World

Scrape the main news for each country.

Do I need a manual approach for each newspaper? Tiresome. Let's try to automate things.
Grab the text element with the largest font on the main page?

Grab the elements, search for h1...h6, pick the first largest one (and the subtext?)?

No, font size is better.

## Links
https://en.wikipedia.org/wiki/List_of_newspapers_in_Sweden

In [236]:
import pandas as pd
import random
import numpy as np

def create_schedule(seed = 0):
    """This function uses a horrible why-loop with a try clause, instead of intelligence. Beware"""
    np.random.seed(seed)
    schedule = pd.DataFrame()
    times = [
    "1400-1420",
    "1425-1445",
    "1450-1510",
    "1515-1535",
    "1540-1600"
    ]

    arenas = ["MONSTER", "LANTCHIPS", "SKÄRGÅRD"]

    A = "Maria/Robert"
    B = "Alexander/Jenny"
    C = "Maria/Niclas"
    D = "Anders/Therese"
    E = "Linnea/Tobias"
    F = "Filip/Victor"

    teams = [A, B, C, D, E, F]

    done = False
    while done is not True:
        try:
            vs_table = pd.DataFrame(columns=['Team_1', 'Team_2'])

            for timeslot in times:
                players = teams.copy()
                for arena in arenas:
                    team1 = np.random.choice(players)
                    players.remove(team1)

                    already_played_against = list(vs_table[vs_table['Team_1'] == team1]['Team_2'].unique())

                    if len(already_played_against) == len(teams) -1:
                        print(f"{team1}  is done!")
                    else:
                        not_played_against = [p for p in players if p not in already_played_against]
            

                        team2 = np.random.choice(not_played_against)
                        players.remove(team2)

                        vs_entry = pd.DataFrame([[team1, team2], [team2, team1]], columns=['Team_1', 'Team_2'])
                        vs_table = vs_table.append(vs_entry)

                        #print(f"{team1} vs {team2} in {arena} at {timeslot}")
                        schedule.at[timeslot, arena] = team1 + ' vs ' + team2
            done = True
        except Exception as e:
            done = False
    return schedule




schedule  = create_schedule(2)

schedule

,MONSTER,LANTCHIPS,SKÄRGÅRD
1400-1420,Maria/Robert vs Alexander/Jenny,Linnea/Tobias vs Filip/Victor,Anders/Therese vs Maria/Niclas
1425-1445,Maria/Robert vs Filip/Victor,Anders/Therese vs Linnea/Tobias,Maria/Niclas vs Alexander/Jenny
1450-1510,Anders/Therese vs Alexander/Jenny,Filip/Victor vs Maria/Niclas,Linnea/Tobias vs Maria/Robert
1515-1535,Maria/Niclas vs Maria/Robert,Alexander/Jenny vs Linnea/Tobias,Filip/Victor vs Anders/Therese
1540-1600,Anders/Therese vs Maria/Robert,Linnea/Tobias vs Maria/Niclas,Alexander/Jenny vs Filip/Victor


In [2]:
import logging
logging.basicConfig(level = logging.INFO)

import psycopg2
import requests
from bs4 import BeautifulSoup
from datetime import datetime



from eyes_of_argus import *

ARGUS_EYES = [
    eye_swe_1(),
    eye_swe_2(),
    eye_swe_3(),
    eye_swe_4(),
    eye_swe_5(),
    eye_den_1()
]
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
translator.translate("Der Himmel ist blau und ich mag Bananen", dest='en').text

'Der Himmel ist blau und ich mag Bananen'

In [3]:
class mind_of_argus():
    """
    The Mind of Argus processes what all eyes see...
    This class takes in a destination language and a list of eyes.
    
    """
    def __init__(self, target_lang = 'en'):
        self.target_lang = target_lang
        self.translator = Translator()
        self.eyes = ARGUS_EYES

    def understand(self, eye):
        """
        Gets the header and extract from the eye and tries to translate it
        """
        response = eye.watch()
        for key in response:
            try:
                text = response[key]
                translation = self.translator.translate(text, dest = self.target_lang).text
                if text == translation:
                    response[key] = "(Translation failed) " + translation
                else:
                    response[key] = cb[4] + translation
            except Exception as e:
                print("e")
                logging.warning(f"Translation failed {e}")
       
        return response

    def convey(self, eye):
        eye.infer_data()
        description = eye.describe()
        print(description['newspaper'])
        response = self.understand(eye)
        for key in response:
            text = response[key]
            print(f"{key}: {text}")

    def overwatch(self):
        for eye in self.eyes:
            print()
            self.convey(eye)



In [5]:

class eye_tha_1(Proto_eye):
    def __init__(self):
        super().__init__(self)
        self.url ='https://www.dailynews.co.th/'
ARGUS_EYES.append(eye_tha_1())



class eye_fra_1(Proto_eye):
    def __init__(self):
        super().__init__(self)
        self.url = 'https://www.lemonde.fr/'
ARGUS_EYES.append(eye_fra_1())

class eye_ger_1(Proto_eye):
    def __init__(self):
        super().__init__(self)
        self.url = 'https://www.bild.de/'
ARGUS_EYES.append(eye_ger_1())


a = mind_of_argus()
a.convey(a.eyes[-1])


(Guess) bild
header: (Translation failed) (Guess) Aktuelle Nachrichten
excerpt: (Translation failed) (Guess) Alle Nachrichten zu Covid-19


'JUST NU: 174 nya dödsfall i Sverige'

{'url': 'https://www.gp.se/',
 'newspaper': '(Guess) gp',
 'country': '(Guess) Sweden',
 'language': '(Not set)',
 'notes': '(Not set)'}

'Der Himmel ist blau und ich mag Bananen'

In [7]:
cb[1]

'(Not set)'

In [48]:


# Add other eyes

# Setup website

# Save daily state in database
# Make it pretty
# Add history
# Search for country mentions
# Add all countries

https://politiken.dk/
Not_set
prototype
header:   Not Found
excerpt:   Not Found
